In [8]:
import os
import keras
import numpy as np
import scipy.signal
from skimage import io, exposure, img_as_uint, img_as_float
import scipy.misc
from scipy.misc import imsave
import scipy.ndimage as ndimage
from scipy.ndimage.filters import gaussian_filter
import math
import glob
from PIL import Image
from decimal import * 
import cv2
import sys
import numexpr as ne
import matplotlib.pyplot as plt
from timeit import default_timer as timer
from keras import backend as K
    

In [18]:
def save_feature(feature, image_list, i, j):

    im = np.array(feature, dtype='float64')
    im = exposure.rescale_intensity(im, out_range='float')
    im = img_as_uint(im)

    image_list.append(im)

In [19]:
def load_model(model_name, image_shape, include_top=True):
    # Load wanted model (resnet, vgg16 or vgg19)

    # set params
    weights = 'imagenet'
    model = None

    if model_name == 'vgg19':
        model = keras.applications.vgg19.VGG19(
            include_top=include_top,
            weights=weights,
            input_shape=image_shape
        )
    else:
        print("Model name is unknown")
    return model

In [20]:
def show_layers(model):

    # Show all layers of model
    print("Layers :")
    for layer in model.layers:
        print("  %s\t--> %s" % (layer.name, layer.output.shape))

In [21]:
def load_image(image_name, image_shape=None):

    image = cv2.imread(image_name).astype(np.float32)
    if image_shape != None:
        image = cv2.resize(image, image_shape)
        
    # Remove train image mean (imagenet)
    image[:,:,0] -= 103.939
    image[:,:,1] -= 116.779
    image[:,:,2] -= 123.68
    image = np.expand_dims(image, axis=0)
    return image


In [22]:
def extract_features(model, image, n, feature_list, num_layer):

    # Try to extract every layer features
    inp = model.input #input placeholder
    outputs = [layer.output for layer in model.layers] # all layer outputs
    functor = K.function([inp]+[K.learning_phase()], outputs )  # evaluation function
    
    # Forward the network
    layer_outs = functor([image, 1.])
    
    for i,features in enumerate(layer_outs[int(num_layer):]): #select 2 last layers
        features = features[0] 
        f_range = range(0, features.shape[2])

        for j in f_range: #iterate on the features of the layer
            image = features[:,:,j]
            save_feature(image, feature_list, i, j)

In [40]:
def bilinear_rescale(feature_list):

    feature_list_resized = []
    
    width, heigth = feature_list[int(len(feature_list)/2)].shape
    final_size = [width,heigth]

    new_im = Image.new('RGB', (final_size[0], final_size[1]))

    i = 0

    for f in feature_list:
        new_im = scipy.misc.imresize(f, (final_size[0],final_size[1]), 'bilinear')
        feature_list_resized.append(new_im)
        i += 1

    return feature_list_resized

In [62]:
def linear_combination(feature_list):

    weight = 1/float(len(feature_list))
    i = 0

    for feature in feature_list:

        if i == 0:
            img = feature 
            final_image = ne.evaluate('img * weight')
            i += 1

        if i < len(feature_list):
            img = feature 
            final_image = final_image + ne.evaluate('img * weight')
            i+=1

    return final_image 

In [48]:
def softmax(im):

    im = Image.open(im)

    pixels = list(im.getdata())
    width, height = im.size
    pixels = [pixels[i * width:(i + 1) * width] for i in range(height)]

    sum = 0

    final = pixels

    #normalize by standard deviation
    std_pixel = pixels / np.std(pixels)

    #sum of exp
    for p in std_pixel:
        for n in p:
            sum += math.exp(n)

    #final array
    i = 0
    for p in std_pixel:
        j=0
        for n in p:
            final[i][j]=math.exp(n)/sum
            j += 1
        i += 1

    max = 0 

    i = 0
    for p in final:
        j=0
        for n in p:
            if(n>max):
                max = n
            j += 1
        i += 1

    i = 0
    for p in std_pixel:
        j=0
        for n in p:
            final[i][j]=(final[i][j]/max)*255
            j += 1
        i += 1

    array = np.array(final, dtype=np.uint8)
    new_image = Image.fromarray(array)

    return new_image

In [43]:
def bias(img,width,heigth):

    bias = scipy.misc.imread('bias.png','L')

    final_size = [width,heigth]
    bias = scipy.misc.imresize(bias,(final_size[0],final_size[1]), 'bilinear')

    img = img * bias 
    return img

In [65]:
sig = 1
layer = 15
m = "vgg19"
input_image = "D:\\ESIR\\ESIR3\\CAV\\BenchmarkIMAGES\\i96.jpg"
image_name = input_image.split('\\')[-1]
start = timer()
image = load_image(input_image)
model = load_model(m,image.shape[1:4],False)

feature_list = []

#feature extraction
extract_features(model,image,0, feature_list, layer)

width, heigth = image.shape[1:3]
final_size = [width,heigth]

width_bis, heigth_bis = feature_list[int(len(feature_list)/2)].shape

#feature rescaling
feature_list = bilinear_rescale(feature_list)

#uniform linear combination
final = linear_combination(feature_list)
feature_list = []

#gaussian filtering
final = gaussian_filter(final,sigma=int(sig))

#applying center bias applying
final = bias(final,width_bis,heigth_bis)
scipy.misc.imsave('..\\out\\saliency\\bias_map.jpg',final)

#softmax funciton
final = softmax('..\\out\\saliency\\bias_map.jpg')
smap = scipy.misc.imresize(final, (width,heigth), 'bilinear')
end = timer()
time = end - start
scipy.misc.imsave('..\\out\\saliency\\'+(image_name.split('.')[0]).split('/')[-1]+'_sig'+str(sig)+'_layer'+str(layer)+'_'+m+'_map.jpg',smap)
os.system('del D:\\perso\\technologie\\projects\\out\\saliency\\bias_map.jpg')

print('Saliency map computed in '+str(time)+' sec')	



C:\Users\charl\Anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\charl\Anaconda3\lib\site-packages\skimage\exposure\exposure.py:297: RuntimeWarning: invalid value encountered in true_divide
  image = (image - imin) / float(imax - imin)
C:\Users\charl\Anaconda3\lib\site-packages\numpy\core\_methods.py:29: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims)
C:\Users\charl\Anaconda3\lib\site-packages\numpy\core\_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)
C:\Users\charl\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  del sys.path[0]


Saliency map computed in 14.628644114350323 sec


C:\Users\charl\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\charl\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  
C:\Users\charl\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
C:\Users\charl\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
C:\Users\charl\Anaconda3\lib\sit